# Data Cleaning and Feature Engineering

In [2]:
import pandas as pd
import numpy as np

ds = pd.read_csv('../data/2023_selected_stocks.csv', low_memory=False)
ds.head()

,date,ticker,name,open,close,min,max,avg,quantity,volume,currency,marketType,bdiCode,prazoT,paperSpecification,optionPrice,priceCorrection,paperDueDate,quoteFactor
0,2023-01-02,ABEV3,AMBEVS/A,14.40,14.19,14.02,14.45,14.19,14995900,2.128569e+08,R$,10,2,,ON,0.0,0,99991231,1
1,2023-01-02,BBDC4,BRADESCO,14.90,14.75,14.66,14.99,14.74,24748300,3.650038e+08,R$,10,2,,PN N1,0.0,0,99991231,1
2,2023-01-02,ITUB4,ITAUUNIBANCO,24.43,24.49,24.04,24.53,24.26,23340700,5.664208e+08,R$,10,2,,PN EJ N1,0.0,0,99991231,1
3,2023-01-02,PETR4,PETROBRAS,23.54,22.92,22.80,23.81,23.09,78424700,1.811379e+09,R$,10,2,,PN N2,0.0,0,99991231,1
4,2023-01-02,VALE3,VALE,88.68,89.40,88.53,89.90,89.42,12783800,1.143138e+09,R$,10,2,,ON NM,0.0,0,99991231,1


In [3]:
ds.drop(columns=['currency','marketType','bdiCode','prazoT','paperSpecification','optionPrice', 'priceCorrection','paperDueDate', 'quoteFactor'], inplace=True)
ds.head()

,date,ticker,name,open,close,min,max,avg,quantity,volume
0,2023-01-02,ABEV3,AMBEVS/A,14.40,14.19,14.02,14.45,14.19,14995900,2.128569e+08
1,2023-01-02,BBDC4,BRADESCO,14.90,14.75,14.66,14.99,14.74,24748300,3.650038e+08
2,2023-01-02,ITUB4,ITAUUNIBANCO,24.43,24.49,24.04,24.53,24.26,23340700,5.664208e+08
3,2023-01-02,PETR4,PETROBRAS,23.54,22.92,22.80,23.81,23.09,78424700,1.811379e+09
4,2023-01-02,VALE3,VALE,88.68,89.40,88.53,89.90,89.42,12783800,1.143138e+09


In [ ]:
## this retrieves the everyday closing information from bovespa
import yfinance as yf

# IBOVESPA ticker on Yahoo Finance
ibov = "^BVSP"

# Download 2023 data
df_ibov = yf.download(ibov, start="2023-01-01", end="2023-11-20")

# Keep only date and close
df_ibov_close = df_ibov[['Close']].reset_index()
df_ibov_close.rename(columns={'Close': 'close'}, inplace=True)

# Save locally
df_ibov_close.to_csv("../data/ibovespa_2023.csv", index=False)

print(df_ibov_close.head())

In [4]:
bovespaDf = pd.read_csv('../data/ibovespa_2023.csv', low_memory=False)

# changing uppercase "Date" to "date"
bovespaDf = bovespaDf.rename(columns={'Date': 'date'})

# renaming to avoid duplicates in "close" variable
bovespaDf = bovespaDf.rename(columns={'close': 'ibovespa_close'})
bovespaDf.head()

,date,ibovespa_close
0,NaN,^BVSP
1,2023-01-02,106376.0
2,2023-01-03,104166.0
3,2023-01-04,105334.0
4,2023-01-05,107518.0


In [5]:
# making sure the date formats are the same
bovespaDf['date'] = pd.to_datetime(bovespaDf['date'])
ds['date'] = pd.to_datetime(ds['date'])

# merge the dataset for bovespa's closing on the date column
ds = ds.merge(bovespaDf[['date', 'ibovespa_close']], on='date', how='left')

ds.tail()


,date,ticker,name,open,close,min,max,avg,quantity,volume,ibovespa_close
1095,2023-11-17,ABEV3,AMBEVS/A,13.71,13.60,13.52,13.74,13.56,36788200,4.991464e+08,125062.0
1096,2023-11-17,BBDC4,BRADESCO,15.57,15.43,15.35,15.60,15.45,34493400,5.331879e+08,125062.0
1097,2023-11-17,VALE3,VALE,74.23,74.24,74.10,74.65,74.35,24001500,1.784705e+09,125062.0
1098,2023-11-17,PETR4,PETROBRAS,35.83,36.71,35.82,36.92,36.53,85895400,3.138326e+09,125062.0
1099,2023-11-17,ITUB4,ITAUUNIBANCO,30.29,30.45,30.12,30.59,30.45,24575400,7.483268e+08,125062.0


## Feature

- Day of the week : different days of the week may influence if a stock's price change, eg Monday & Fridays more agitated
- daily_return = (close - open) / open  - Simple return rate %
- price_range = max - min - Intraday volatility
- volume_per_quantity = volume / quantity - Trade size indicator

We will add more features to

In [10]:
## Features
# - Day of the week : different days of the week may influence if a stock's price change, eg Monday & Fridays more agitated
# - daily_return = (close - open) / open  - Simple return rate %
# - price_range = max - min - Intraday volatility
# - volume_per_quantity = volume / quantity - Trade size indicator

import pandas as pd

# Convert date column to datetime if not already
ds["date"] = pd.to_datetime(ds["date"])

# --- Feature Engineering ---

# 1. Day of the week (Monday=0, Sunday=6)
#ds["day_of_week"] = ds["date"].dt.day_name()  # for string names
ds["day_of_week"] = ds["date"].dt.weekday  # for numeric

# 2. Daily return = (close - open) / open
ds["daily_return"] = (ds["close"] - ds["open"]) / ds["open"]

# 3. Price range = max - min
ds["price_range"] = ds["max"] - ds["min"]

# 4. Volume per quantity = volume / quantity
ds["volume_per_quantity"] = ds["volume"] / ds["quantity"]

         date ticker          name   open  close    min    max    avg  \
50 2023-01-16  BBDC4      BRADESCO  14.71  14.53  14.44  14.80  14.58   
51 2023-01-16  PETR4     PETROBRAS  24.41  24.04  23.83  24.50  24.08   
52 2023-01-16  VALE3          VALE  92.00  92.25  90.83  92.39  91.75   
53 2023-01-16  ITUB4  ITAUUNIBANCO  25.75  25.57  25.29  25.85  25.48   
54 2023-01-16  ABEV3      AMBEVS/A  13.72  13.20  13.18  13.74  13.36   

    quantity        volume  ... daily_return  price_range  \
50  44231400  6.450610e+08  ...    -0.012237         0.36   
51  31973700  7.700440e+08  ...    -0.015158         0.67   
52  19563900  1.795009e+09  ...     0.002717         1.56   
53  24481400  6.240123e+08  ...    -0.006990         0.56   
54  58837000  7.863234e+08  ...    -0.037901         0.56   

    volume_per_quantity  target  rolling_volume  rolling_close_5  \
50            14.583781   14.63    1.289860e+09           15.018   
51            24.083669   25.52    1.201422e+09           

,date,ticker,name,open,close,min,max,avg,quantity,volume,...,daily_return,price_range,volume_per_quantity,target,rolling_volume,rolling_close_5,rolling_std_5,rolling_return_5,rolling_volume_5,momentum_5
1085,2023-11-14,VALE3,VALE,72.02,73.61,71.90,74.30,73.50,52233600,3.839284e+09,...,0.022077,2.40,73.502195,74.10,1.145008e+09,70.726,0.702019,-0.003421,1.573015e+09,0.040777
1086,2023-11-14,ITUB4,ITAUUNIBANCO,29.49,29.83,29.43,30.07,29.78,41826500,1.245616e+09,...,0.011529,0.64,29.780535,30.31,9.161853e+08,29.262,0.238265,0.007318,8.726402e+08,0.019411
1087,2023-11-14,BBDC4,BRADESCO,14.80,15.05,14.71,15.09,14.94,42559800,6.361136e+08,...,0.016892,0.38,14.946348,15.57,7.772947e+08,15.084,0.210309,0.006352,8.210040e+08,-0.002254
1088,2023-11-14,ABEV3,AMBEVS/A,13.50,13.70,13.45,13.75,13.66,24478100,3.345021e+08,...,0.014815,0.30,13.665362,13.67,6.234196e+08,13.460,0.092195,-0.002646,2.719510e+08,0.017831
1089,2023-11-14,PETR4,PETROBRAS,35.69,36.18,35.49,36.24,35.88,50888600,1.826352e+09,...,0.013729,0.75,35.889207,35.55,7.906573e+08,34.876,0.544821,0.005033,1.733072e+09,0.037390


## Leakage Guards

In [ ]:
# 5. Close of previous day
# This aligns today’s features with tomorrow’s close.
ds['target'] = ds.groupby('ticker')['close'].shift(-1)

In [ ]:
# 6. Average of the previous 5 days of volume for each stock
# That .shift(1) is critical — it prevents future leakage.
ds['rolling_volume'] = ds.groupby('ticker')['volume'].shift(1).rolling(5).mean()

## Rolling features


In [ ]:
# --- Rolling features ---
ds['rolling_close_5']   = ds.groupby('ticker')['close'].transform(lambda x: x.shift(1).rolling(5).mean())
ds['rolling_std_5']     = ds.groupby('ticker')['close'].transform(lambda x: x.shift(1).rolling(5).std())
ds['rolling_return_5']  = ds.groupby('ticker')['daily_return'].transform(lambda x: x.shift(1).rolling(5).mean())
ds['rolling_volume_5']  = ds.groupby('ticker')['volume'].transform(lambda x: x.shift(1).rolling(5).mean())
ds['momentum_5']        = ds['close'] / ds['rolling_close_5'] - 1
# --- Optional: handle infinities or NaNs ---
ds.replace([float("inf"), float("-inf")], pd.NA, inplace=True)
ds.dropna(subset=["daily_return", "price_range", "volume_per_quantity"], inplace=True)
# --- Drop NaNs from rolling features ---
ds.dropna(subset=[
    'rolling_close_5', 'rolling_std_5', 'rolling_return_5',
    'momentum_5', 'rolling_volume_5', 'target'
], inplace=True)

# --- View results ---
ds.tail()

In [ ]:
# --- Save to CSV ---
output_path = "../data/2023_stock_with_features.csv"
ds.to_csv(output_path, index=False)
ds.tail()